# Операции над данными в библиотеке Pandas

## Универсальные функции

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns = ['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Если применить универсальную функцию NumPy к любому из этих объектов, результатом будет другой объект библиотеки Pandas с сохранением индексов:

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## Универсальные функции: выравнивание индексов

### Выравнивание индексов в объекте Series

При бинарных операциях над двумя объектами Series или DataFrame библиотека Pandas будет выравнивать индексы в процессе выполнения операции.

In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,'New York': 19651127}, name='population')

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [8]:
# получившийся в итоге объект содержит объединение индексов двух исходных массивов, 
# которое можно определить посредством стандартной арифмерики множеств языка Pyton для этих индексов:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Аналогичным образом реализовано сопоставление индексов для всех встроенных арифметических выражений языка Python: все отсутствующие значения заполняются по умолчанию значением NaN:

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [10]:
# можно избавиться от объектов NaN, задав заполнитеь явно для любых потенциально отсутствующих элементов в объектах A или B
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Выравнивание индексов в объектах DataFrame

При выполнении операций над объектами DataFrame происходит аналогичное выравнивание как для столбцов, так и для индексов

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [13]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


In [14]:
B + A

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


следующем примере мы заполним отсутствующие значения средним значением всех элементов объекта A (которое вычислим, выстроив сначала значения объекта A в один столбец с помощью функции stack):

In [17]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


## Универсальные функции: выполнение операции между объектами DataFrame и Series

Операции между объектами DataFrame и Series подобны операциям между двумерным и одномерным массивами библиотеки NumPy.

In [18]:
# одна из часто встречающихся операций - вычисление разности двумерного массива и одной из его строк

In [19]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [22]:
# в соответствии с правилами транслирования библиотеки NumPy, 
# вычитание из двумерного массива одной из его строк выполняется построчно.
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

In [24]:
# в библиотеке Pandas вычитание по умолчанию также происходит построчно
df = pd.DataFrame(A, columns=list('QRST'))
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [29]:
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


In [33]:
# можно выполнить эту операцию по столбцам
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Операции DataFrame/Series будут автоматически выполнять выравнивание индексов между двумя их элементами

In [34]:
halfraw = df.iloc[0, ::2]
halfraw

Q    3
S    2
Name: 0, dtype: int64

In [35]:
df - halfraw

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


Подобное сохранение и выравнивание индексов и столбцов означает, что операции над данными в библиотеке Pandas всегда сохраняют контекст данных, предотвращая возможные ошибки при работе с неоднородными и/или неправильно/неодинаково выровненными данными в исходных массивах NumPy.